In [3]:
import numpy as np
import math
import os
import bucket_util as bu
import datetime

In [4]:
currdir = 'D:\itd\cleandata'+'\data_output_full_'
outputdir = 'D:\itd\histdata'+'\data_output_full_'

# load yield data
data_yield = np.genfromtxt('yeild.csv', delimiter=',')
locations = np.genfromtxt('loc.csv', delimiter=',')
        
# generate index for all data
length = data_yield.shape[0]
index_all = np.arange(length)

In [5]:
 def filter_timespan(image_temp,start_day,end_day,bands):
        start_index=int(math.floor(start_day/8))*bands
        end_index=int(math.floor(end_day/8))*bands
        if end_index>image_temp.shape[2]:
            image_temp = np.concatenate((image_temp, 
                np.zeros((image_temp.shape[0],image_temp.shape[1],end_index-image_temp.shape[2]))),axis=2)
        return image_temp[:,:,start_index:end_index]

In [6]:
def calc_histogram(image_temp,bin_seq,bins,times,bands):
        hist=np.zeros([bins,times,bands])
        for i in range(image_temp.shape[2]):
            density, _ = np.histogram(image_temp[:, :, i], bin_seq, density=False)
            if density.sum()==0: # WE UNCOMMENTED THIS BC SOME IMAGES ARE ALL 0S!!!
                 continue
            hist[:, i // bands, i % bands] = density / float(density.sum())
        return hist

In [5]:
np.histogram([1,2,3,2,3],bins=[1,2,3])

(array([1, 4], dtype=int64), array([1, 2, 3]))

In [8]:
output_image = np.zeros([index_all.shape[0], 32, 32, 9])
output_yield = np.zeros([index_all.shape[0]])
output_year = np.zeros([index_all.shape[0]])
output_locations = np.zeros([index_all.shape[0],2])
output_index = np.zeros([index_all.shape[0],2])

In [9]:
index_all

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [8]:
data_yield[0]

array([2.015e+03, 1.000e+01, 1.000e+00, 4.250e+01])

In [9]:
year = str(int(data_yield[0, 0]))
loc1 = str(int(data_yield[0, 1]))
loc2 = str(int(data_yield[0, 2]))

In [10]:
key = np.array([int(loc1),int(loc2)])

In [11]:
index = np.where(np.all(locations[:,0:2].astype('int') == key, axis=1))
longitude = locations[index,2]
latitude = locations[index,3]

In [12]:
filename = year + '_' + loc1 + '_' + loc2 + '.npy'
print(currdir+filename)
image_temp = np.load(currdir + filename)
image_temp.shape

D:\itd\cleandata\data_output_full_2015_10_1.npy


(120, 103, 414)

In [13]:
image_temp = filter_timespan(image_temp, 49, 305, 9)

In [14]:
image_temp.shape

(120, 103, 288)

In [15]:
bin_seq = np.linspace(1, 4999,33)
bin_seq

array([1.0000000e+00, 1.5718750e+02, 3.1337500e+02, 4.6956250e+02,
       6.2575000e+02, 7.8193750e+02, 9.3812500e+02, 1.0943125e+03,
       1.2505000e+03, 1.4066875e+03, 1.5628750e+03, 1.7190625e+03,
       1.8752500e+03, 2.0314375e+03, 2.1876250e+03, 2.3438125e+03,
       2.5000000e+03, 2.6561875e+03, 2.8123750e+03, 2.9685625e+03,
       3.1247500e+03, 3.2809375e+03, 3.4371250e+03, 3.5933125e+03,
       3.7495000e+03, 3.9056875e+03, 4.0618750e+03, 4.2180625e+03,
       4.3742500e+03, 4.5304375e+03, 4.6866250e+03, 4.8428125e+03,
       4.9990000e+03])

In [16]:
image_temp = calc_histogram(image_temp, bin_seq ,32, 32, 9)

In [17]:
image_temp.shape

(32, 32, 9)

In [18]:
image_temp[:,1,1]

array([0.        , 0.        , 0.00303951, 0.        , 0.00303951,
       0.0668693 , 0.01823708, 0.00607903, 0.00303951, 0.00911854,
       0.03647416, 0.0668693 , 0.0881459 , 0.09726444, 0.15197568,
       0.0881459 , 0.0881459 , 0.07294833, 0.02431611, 0.0212766 ,
       0.03647416, 0.02431611, 0.02431611, 0.00911854, 0.00303951,
       0.01215805, 0.01215805, 0.00303951, 0.00911854, 0.01519757,
       0.00303951, 0.00303951])

In [19]:
image_temp.sum()

288.0

In [20]:
output_image[0, :] = image_temp
output_yield[0] = data_yield[0, 3]
output_year[0] = int(year)
output_locations[0, 0] = longitude
output_locations[0, 1] = latitude
output_index[0,:] = np.array([int(loc1),int(loc2)])

In [21]:
np.savez(outputdir+'histogram_all_full.npz',
                 output_image=output_image,output_yield=output_yield,
                 output_year=output_year,output_locations=output_locations,output_index=output_index)

In [ ]:
for i in index_all:
            year = str(int(data_yield[i, 0]))
            loc1 = str(int(data_yield[i, 1]))
            loc2 = str(int(data_yield[i, 2]))

            key = np.array([int(loc1),int(loc2)])
            index = np.where(np.all(locations[:,0:2].astype('int') == key, axis=1))
            longitude = locations[index,2]
            latitude = locations[index,3]

            print (datetime.datetime.now())
            filename = year + '_' + loc1 + '_' + loc2 + '.npy'
            print ("Examining file: %s!" % filename)
            try:
                # This is misleading - it's not just temperature bands - it's the
                # entire multispectral image of depth 9 * times for a single
                # year at a particular location.
                print(currdir+filename)
                image_temp = np.load(currdir + filename)
                # print("1")
                # print (image_temp.shape)
                # print(np.count_nonzero(image_temp))
                # Filter the image depth to only have the selected days.
                # Days 49-305 are shown here (9 bands each time), but in the paper,
                # the researchers use 49-281.
                image_temp = filter_timespan(image_temp, 49, 305, 9)
                # print("2")
                # print (datetime.datetime.now())
                # print (image_temp.shape)
                # print(np.count_nonzero(image_temp))

                bin_seq = np.linspace(1, 4999, 33)
                image_temp = calc_histogram(image_temp, bin_seq ,32, 32, 9) # ?
                # print("3")
                # print (image_temp.shape)
                # print(np.count_nonzero(image_temp))
                image_temp[np.isnan(image_temp)] = 0
                print("sum:")
                # print (image_temp.shape)
                # print(np.count_nonzero(image_temp))
                print(np.sum(image_temp))
                if np.sum(image_temp) < 250:
                    print ('broken image', filename)

                output_image[i, :] = image_temp
                output_yield[i] = data_yield[i, 3]
                output_year[i] = int(year)
                output_locations[i, 0] = longitude
                output_locations[i, 1] = latitude
                output_index[i,:] = np.array([int(loc1),int(loc2)])

                print (i,np.sum(image_temp),year,loc1,loc2)
            except IOError:
                print ("File: %s not found!" % filename)
                continue
            except ValueError:
                print ("File: Value error in %s!" % filename)
                continue
np.savez(outputdir+'histogram_all_full.npz',
                 output_image=output_image,output_yield=output_yield,
                 output_year=output_year,output_locations=output_locations,output_index=output_index)
print ('save done')

2018-04-23 10:28:47.126423
Examining file: 2015_10_1.npy!
D:\itd\cleandata\data_output_full_2015_10_1.npy
sum:
287.99999999999994
0 287.99999999999994 2015 10 1
2018-04-23 10:28:47.274908
Examining file: 2014_10_1.npy!
D:\itd\cleandata\data_output_full_2014_10_1.npy
sum:
288.0
1 288.0 2014 10 1
2018-04-23 10:28:48.052391
Examining file: 2013_10_1.npy!
D:\itd\cleandata\data_output_full_2013_10_1.npy
sum:
288.0
2 288.0 2013 10 1
2018-04-23 10:28:48.667752
Examining file: 2012_10_1.npy!
D:\itd\cleandata\data_output_full_2012_10_1.npy
sum:
288.00000000000006
3 288.00000000000006 2012 10 1
2018-04-23 10:28:49.229170
Examining file: 2011_10_1.npy!
D:\itd\cleandata\data_output_full_2011_10_1.npy
sum:
287.99999999999994
4 287.99999999999994 2011 10 1
2018-04-23 10:28:49.809090
Examining file: 2010_10_1.npy!
D:\itd\cleandata\data_output_full_2010_10_1.npy
sum:
288.0
5 288.0 2010 10 1
2018-04-23 10:28:50.551237
Examining file: 2009_10_1.npy!
D:\itd\cleandata\data_output_full_2009_10_1.npy
sum:
2

sum:
288.0
55 288.0 2005 12 59
2018-04-23 10:29:22.646414
Examining file: 2005_12_63.npy!
D:\itd\cleandata\data_output_full_2005_12_63.npy
sum:
288.0
56 288.0 2005 12 63
2018-04-23 10:29:23.348039
Examining file: 2005_12_65.npy!
D:\itd\cleandata\data_output_full_2005_12_65.npy
sum:
288.0
57 288.0 2005 12 65
2018-04-23 10:29:23.949765
Examining file: 2005_12_133.npy!
D:\itd\cleandata\data_output_full_2005_12_133.npy
sum:
288.0
58 288.0 2005 12 133
2018-04-23 10:29:24.503884
Examining file: 2004_12_79.npy!
D:\itd\cleandata\data_output_full_2004_12_79.npy
sum:
288.0
59 288.0 2004 12 79
2018-04-23 10:29:25.104438
Examining file: 2004_12_121.npy!
D:\itd\cleandata\data_output_full_2004_12_121.npy
sum:
288.00000000000006
60 288.00000000000006 2004 12 121
2018-04-23 10:29:25.738473
Examining file: 2004_12_13.npy!
D:\itd\cleandata\data_output_full_2004_12_13.npy
sum:
288.0
61 288.0 2004 12 13
2018-04-23 10:29:26.223708
Examining file: 2004_12_33.npy!
D:\itd\cleandata\data_output_full_2004_12_33